In [ ]:
# INITIALIZE PYQGIS (run once per session)
import sys, os
os.environ['QT_QPA_PLATFORM']='offscreen'
sys.path.append('/apps/share64/debian7/anaconda/anaconda3-5.1/envs/qgis/')
from qgis.core import *
from qgis.analysis import QgsNativeAlgorithms
#from qgis.utils import *
import processing
from processing.core.Processing import Processing 

qgs = QgsApplication([], False)
qgs.initQgis()
Processing.initialize()
QgsApplication.processingRegistry().addProvider(QgsNativeAlgorithms()) 

In [ ]:
# IMPORT LIBRARIES
import urllib, math, zipfile, shutil

In [ ]:
# USER INPUTS
input_folder_name = "/home/mygeohub/kpaliwal/CWH/Input"
output_folder_name = "/home/mygeohub/kpaliwal/CWH/Output" 
boundary_file =  "Boundary.shp"

In [ ]:
# USER DEFINED FUNCTIONS

def GetNED(NL, WL):
    name1 = "n"+NL+"w"+WL
    address = "ftp://rockyftp.cr.usgs.gov/vdelivery/Datasets/Staged/Elevation/1/ArcGrid/USGS_NED_1_"
    url_final = address + name1 + "_ArcGrid.zip"
    print(url_final)
    try:
        urllib.request.urlretrieve(url_final, work_folder_name + "/" + name1+".zip")
        print(work_folder_name + "/" + name1 + ".zip")
        print("NED download successful: " + name1)
        #unzip the folder
        zfile1 = zipfile.ZipFile(work_folder_name + "/" + name1 +".zip", 'r')
        zfile1.extractall(work_folder_name)
        zfile1.close()
        print("NED extraction successful: " + name1)
        
    except:
        print("Error in NED download or extraction: " + name1)

    #return the name of downloaded file
    return("grd" + name1 + "_1")

In [ ]:
# MAIN CODE STARTS

work_folder_name = os.path.join(input_folder_name, "Work Folder")
if os.path.exists(work_folder_name) == False:
    os.mkdir(work_folder_name)
boundary_path = os.path.join(input_folder_name, boundary_file)
input_crs = QgsVectorLayer(boundary_path, '', 'ogr' ).crs().authid()
#processing.run('qgis:reprojectlayer',{'INPUT': full_input_path, 'TARGET_CRS':'EPSG:102673','OUTPUT': folder_name + "boundary_proj.shp"})
processing.run('native:reprojectlayer',{'INPUT': boundary_path, 'TARGET_CRS':'EPSG:4326','OUTPUT': work_folder_name + "/boundary_proj.shp"})
ext = QgsVectorLayer(work_folder_name + "/boundary_proj.shp", '', 'ogr' ).extent()
print("Bounds of the input file:")
print(ext.yMinimum())
print(ext.yMaximum())
print(ext.xMinimum())
print(ext.xMaximum())
#print(work_folder_name + "/boundary_proj.shp")
listNL=list(range(int(math.ceil(ext.yMinimum())), int(math.ceil(ext.yMaximum()))+1))
listWL=list(range(int(abs(math.floor(ext.xMaximum()))), int(abs(math.floor(ext.xMinimum())))+1))
raster_names = []

In [ ]:
# Looping through tiles to download

for i in listNL:
    for j in listWL:
        NL = str(i)
        WL = "0"+str(j)
        print("Processing: " + NL + " " + WL)
        raster_names.append(GetNED(NL,WL))

In [ ]:
# GIS operations to get the final DEM

input_list = [os.path.join(work_folder_name, cur_raster) for cur_raster in raster_names]
print("Merging Raster...")
processing.run("gdal:merge", {'INPUT':input_list, 'OUTPUT':work_folder_name + "/merged_rast.tif"})
print("Projecting Raster...")
processing.run('gdal:warpreproject', {'INPUT': work_folder_name + "/merged_rast.tif", 'TARGET_CRS': input_crs, 'OUTPUT': work_folder_name + "/proj_rast.tif"})
print("Clipping Raster...")
processing.run('gdal:cliprasterbymasklayer',{'INPUT': work_folder_name + "/proj_rast.tif", 'MASK': boundary_path, 'OUTPUT': output_folder_name + "/" + boundary_file[:-4] + ".tif"})
print("DEM prepared successfully!!!")

In [ ]:
#Clean Up

shutil.rmtree(work_folder_name)
qgs.exitQgis()